# Ici on refait le test mais en prenant que les titres

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

On commence par charger les trois datasets que nous allons utiliser et on uniformise la labellisation avec une colonne "label"qui vaut 1 s'il s'agit d'une fake news et 0 sinon. On choisit de ne considérer que trois des datasets de l'article car le dernier,de taille beaucoup plus faible, ne permet pas dans l'article de Hoy et Koulouri d'obtenir des résultats corrects même sur une partie nouvelle du dataset.

**Chargement et mise en forme de ISOT**

In [2]:
# Mise en forme dataset ISOT https://www.kaggle.com/datasets/csmalarkodi/isot-fake-news-dataset/
Isot_true_df = pd.read_csv("data/True.csv")
Isot_fake_df = pd.read_csv("data/Fake.csv")

#Création d'un dataset unique

Isot_true_df["label"] = 0  # Vraie news
Isot_fake_df["label"] = 1  # Fake news

Isot_data = pd.concat([Isot_true_df, Isot_fake_df], ignore_index=True)

Isot_data.head(5)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [3]:
# On test la présence de NA
Isot_data.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [4]:
# On ne garde que le label et le title de l'article
Isot = Isot_data[['title', 'label']]
Isot.head(3)

,title,label
0,"As U.S. budget fight looms, Republicans flip t...",0
1,U.S. military to accept transgender recruits o...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,0


**Chargement et mise en forme de fake_news**

In [5]:
# Mise en forme dataset Fake_News https://www.kaggle.com/competitions/fake-news/data?select=train.csv
fake_news_data = pd.read_csv("data/train.csv")
fake_news_data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
# On teste la présence de valeurs manquantes sur le texte, et on drop s'il y en a
fake_news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
fake_news_data = fake_news_data.dropna(subset=['title'])

In [8]:
# On ne garde que le label et le title de l'article
fake_news = fake_news_data[['title', 'label']]
fake_news.head(3)

,title,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1


**Chargement et mise en forme de fake_real**

In [9]:
# Mise en forme dataset Fake_real https://www.kaggle.com/datasets/jillanisofttech/fake-or-real-news
fake_real_data = pd.read_csv("data/fake_or_real_news.csv")
fake_real_data['label'] = fake_real_data['label'].map({'FAKE': 1, 'REAL': 0})
fake_real_data.head(5)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [10]:
# On test la présence de NA
fake_real_data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [11]:
# On ne garde que le label et le title de l'article
fake_real = fake_real_data[['title', 'label']]
fake_real.head(3)

,title,label
0,You Can Smell Hillary’s Fear,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,1
2,Kerry to go to Paris in gesture of sympathy,0


A partir de la lecture de l'article référence, il apparait que sur ce jeu de données il est possible d'atteindre de très bonnes performances avec plusieurs features extraction et modèle de machine learning. On s'appuie donc pour commencer sur une approche semblable au V.E, qui repose sur l'utilisation d'un TF-IDF feature extracteur et une régression logistique (dans le but de relever ensuite les mots utilisés, à l'image de ce qu'ils font dans l'article).

A noter que l'on pourra ensuite imaginer d'autres approches, ou voir si l'on arrive à faire quelque chose de correct avec juste le titre etc...

# Construction d'un modèle "baseline" TF-IDF et régression logistique sur ISOT

### Réalisation du modèle

In [12]:
# On réalise le split des données

X = Isot['title']  # Les articles/textes
y = Isot['label']  # Les labels (1 = Fake)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Vectorisation tfidf (ici probablement des paramètres à fine-tuned, notamment le max_df)

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [14]:
#On applique une régression logistique
model_tfidf_logistic = LogisticRegression()
model_tfidf_logistic.fit(X_train_tfidf, y_train)

LogisticRegression()

In [15]:
# Et on évalue notre petit modèle
y_pred = model_tfidf_logistic.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9451002227171492

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.95      0.94      4330
           1       0.95      0.94      0.95      4650

    accuracy                           0.95      8980
   macro avg       0.94      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980


Confusion Matrix:
 [[4110  220]
 [ 273 4377]]


On trouve un modèle qui est donc particulièrement efficace en ne s'intéressant qu'aux titres des articles (!!)

On peut essayer de tester sur un nouveau dataset

### Test sur les autres datasets

In [16]:
# Définition d'une fonction de prédiction 

def apply_model_tfidf_logistic(new_data, text_column='title'):
    """
    Applique le modèle de détection de fake news entrainé sur ISOT et fonctionnant avec un tokenizer TF-IDF et une régression logistique à un nouveau DataFrame.
    Ajoute une colonne 'prediction' (0 = Real, 1 = Fake).
    """
    # Transformer les textes avec le TF-IDF déjà entraîné
    X_new_tfidf = vectorizer.transform(new_data[text_column])
    
    # Faire les prédictions
    predictions = model_tfidf_logistic.predict(X_new_tfidf)
    
    # Ajouter la colonne 'prediction'
    new_data = new_data.copy()  # Pour ne pas modifier le DataFrame original
    new_data['prediction'] = predictions
    
    return new_data
    

In [17]:
# Appliquer le modèle à Fake_News
fake_news_pred = apply_model_tfidf_logistic(fake_news)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_news_pred['label'], fake_news_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_news_pred['label'], fake_news_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.5926291868392451

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.57      0.59     10387
           1       0.58      0.62      0.60      9855

    accuracy                           0.59     20242
   macro avg       0.59      0.59      0.59     20242
weighted avg       0.59      0.59      0.59     20242


Confusion Matrix:
 [[5876 4511]
 [3735 6120]]


In [18]:
# Appliquer le modèle à Fake_Real
fake_real_pred = apply_model_tfidf_logistic(fake_real)

print("\nÉvaluation sur Fake News :\n")
print("Accuracy:", accuracy_score(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nClassification Report:\n", classification_report(fake_real_pred['label'], fake_real_pred['prediction']))
print("\nConfusion Matrix:\n", confusion_matrix(fake_real_pred['label'], fake_real_pred['prediction']))


Évaluation sur Fake News :

Accuracy: 0.5499605367008682

Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.46      0.51      3171
           1       0.54      0.64      0.59      3164

    accuracy                           0.55      6335
   macro avg       0.55      0.55      0.55      6335
weighted avg       0.55      0.55      0.55      6335


Confusion Matrix:
 [[1456 1715]
 [1136 2028]]


Chose intéressante, on a une légère augmentation de l'accuracy (!!) et surtout on n'a beaucoup moins ce biais de sous-reconnaissance des articles "vrais".